# Mediana con **preservación de bordes** (enfoque práctico)

Este cuaderno compara:
1. **Mediana estándar**.
2. **Mediana *edge-aware*** (personalizada): aplica la mediana solo sobre vecinos cercanos en intensidad.
3. **Bilateral** (preserva-bordes) como referencia.

Se usa una imagen con ruido **sal y pimienta**, y se reportan **PSNR** y **SSIM**.

## Dependencias (opcional)

In [ ]:
import sys, subprocess
for p in ['numpy','matplotlib','scikit-image','scipy','pandas']:
    try:
        __import__(p if p!='scikit-image' else 'skimage')
    except ImportError:
        subprocess.check_call([sys.executable,'-m','pip','install',p])
print('Ready!')

## Cargar imagen y añadir ruido

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.util import random_noise
image = img_as_float(data.camera())
noisy = random_noise(image, mode='s&p', amount=0.1)
plt.figure(figsize=(5,5))
plt.imshow(noisy, cmap='gray')
plt.title('Imagen con ruido sal & pimienta (10%)')
plt.axis('off')
plt.show()

## Filtros

In [ ]:
from skimage.filters import median
from skimage.morphology import square
from skimage.restoration import denoise_bilateral

def edge_aware_median(img, size=7, tol=0.08):
    import numpy as np
    pad = size // 2
    padded = np.pad(img, pad, mode='reflect')
    out = np.empty_like(img)
    H, W = img.shape
    min_valid = max(5, (size*size)//6)
    for i in range(H):
        for j in range(W):
            patch = padded[i:i+size, j:j+size]
            center = padded[i+pad, j+pad]
            mask = np.abs(patch - center) <= tol
            vals = patch[mask]
            if vals.size >= min_valid:
                out[i,j] = np.median(vals)
            else:
                out[i,j] = np.median(patch)
    return out

win = 7
med_std = median(noisy, footprint=square(win))
med_eaw = edge_aware_median(noisy, size=win, tol=0.08)
bilat = denoise_bilateral(noisy, sigma_color=0.08, sigma_spatial=3, channel_axis=None)
for title, im in [('Mediana estándar', med_std), ('Mediana edge-aware', med_eaw), ('Bilateral', bilat)]:
    plt.figure(figsize=(5,5))
    plt.imshow(im, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

## Métricas (PSNR/SSIM)

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import pandas as pd
def metrics(orig, rec):
    return psnr(orig, rec, data_range=1.0), ssim(orig, rec, data_range=1.0)
rows = []
for name, im in [('Mediana estándar', med_std), ('Mediana edge-aware', med_eaw), ('Bilateral', bilat)]:
    p, s = metrics(image, im)
    rows.append({'Filtro': name, 'PSNR': p, 'SSIM': s})
import pandas as pd
df = pd.DataFrame(rows)
df

## Recorte para inspección de bordes

In [ ]:
r0, r1, c0, c1 = 120, 220, 170, 270
crops = {
    'Noisy (recorte)': noisy[r0:r1, c0:c1],
    'Mediana estándar (recorte)': med_std[r0:r1, c0:c1],
    'Mediana edge-aware (recorte)': med_eaw[r0:r1, c0:c1],
    'Bilateral (recorte)': bilat[r0:r1, c0:c1]
}
for title, im in crops.items():
    plt.figure(figsize=(4,4))
    plt.imshow(im, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

## Notas
- La **mediana** ya preserva bordes mejor que la media.
- La **edge-aware** evita cruzar bordes fuertes al excluir vecinos alejados en intensidad.
- **Bilateral** sirve como baseline de preservación de bordes.
- Ajusta `size`, `tol`, `sigma_color` y `sigma_spatial` según la textura/ruido.